In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

In [2]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    return random.uniform(1, 100)  # Example implementation

In [3]:
trades = pd.DataFrame({
    'Date': [datetime(2023, 1, 1), datetime(2023, 1, 2), datetime(2023, 1, 3)],
    'Symbol': ['AAPL', 'GOOGL', 'MSFT'],
    'Side': ['buy', 'sell', 'buy'],
    'Size': [10, 5, 8],
    'Price': [150, 2500, 300]
})

In [4]:
def calculate_trade_performance(trades: pd.DataFrame) -> pd.Series:
    # Handle potential edge cases
    if trades.empty:
        return pd.Series({})

    # Fill missing values
    trades['Size'] = trades['Size'].fillna(1)

    # Sort trades by date
    trades = trades.sort_values(by='Date')

    # Initialize metrics
    metrics = {
        'Total Profit/Loss': 0,
        'Average Profit/Loss per Trade': 0,
        'Win Rate': 0,
        'Loss Rate': 0,
        'Maximum Drawdown': 0,
        'Sharpe Ratio': 0,
        'Sortino Ratio': 0,
        'Profit Factor': 0,
        'Trade Duration': 0,
        'Annualized Return': 0
    }

    # Calculate the necessary values
    profits = []
    returns = []
    durations = []

    for i, trade in trades.iterrows():
        # Get the current market price
        current_price = getTickerPrice(trade['Symbol'], trade['Date'])

        # Calculate profit/loss
        if trade['Side'] == 'buy':
            profit = (current_price - trade['Price']) * trade['Size']
        else:  # 'sell'
            profit = (trade['Price'] - current_price) * trade['Size']

        profits.append(profit)
        returns.append(profit / (trade['Price'] * trade['Size']))
        durations.append(1)  # Assuming 1 day duration for simplicity

    # Total Profit/Loss
    metrics['Total Profit/Loss'] = sum(profits)

    # Average Profit/Loss per Trade
    metrics['Average Profit/Loss per Trade'] = np.mean(profits)

    # Win Rate
    win_trades = [p for p in profits if p > 0]
    metrics['Win Rate'] = len(win_trades) / len(profits)

    # Loss Rate
    loss_trades = [p for p in profits if p < 0]
    metrics['Loss Rate'] = len(loss_trades) / len(profits)

    # Maximum Drawdown
    cumulative_profit = np.cumsum(profits)
    rolling_max = np.maximum.accumulate(cumulative_profit)
    drawdowns = rolling_max - cumulative_profit
    metrics['Maximum Drawdown'] = np.max(drawdowns)

    # Sharpe Ratio
    returns = np.array(returns)
    metrics['Sharpe Ratio'] = np.mean(returns) / np.std(returns) * np.sqrt(252)  # Assuming daily returns

    # Sortino Ratio
    downside_returns = returns[returns < 0]
    metrics['Sortino Ratio'] = np.mean(returns) / np.std(downside_returns) * np.sqrt(252)

    # Profit Factor
    gross_profit = sum(p for p in profits if p > 0)
    gross_loss = -sum(p for p in profits if p < 0)
    metrics['Profit Factor'] = gross_profit / gross_loss if gross_loss != 0 else np.nan

    # Trade Duration
    metrics['Trade Duration'] = np.mean(durations)

    # Annualized Return
    total_return = cumulative_profit[-1]
    total_days = (trades['Date'].max() - trades['Date'].min()).days
    metrics['Annualized Return'] = (1 + total_return) ** (252 / total_days) - 1  # Assuming 252 trading days in a year

    return pd.Series(metrics)

In [5]:
performance_metrics = calculate_trade_performance(trades)
print(performance_metrics)

Total Profit/Loss                9346.617269
Average Profit/Loss per Trade    3115.539090
Win Rate                            0.333333
Loss Rate                           0.666667
Maximum Drawdown                 1919.552182
Sharpe Ratio                       -2.565091
Sortino Ratio                     -15.681272
Profit Factor                       4.415286
Trade Duration                      1.000000
Annualized Return                        inf
dtype: float64


<ipython-input-4-f7449644f166>:84: RuntimeWarning: overflow encountered in scalar power
  metrics['Annualized Return'] = (1 + total_return) ** (252 / total_days) - 1  # Assuming 252 trading days in a year
